# Подход 1: градиентный бустинг "в лоб"

In [1]:
import pandas as pd
import numpy as np
#1. Чтение таблицы с признаками
features = pd.read_csv('features.csv', index_col = 'match_id')
features.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
#Удаление из таблицы признаков, связанных с итогами матча
features_edit = features.drop(columns = ['duration',
                             'radiant_win',
                             'tower_status_radiant',
                             'tower_status_dire',
                             'barracks_status_dire',
                             'barracks_status_radiant'])
features_edit.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


#### 2.Проверка выборки на наличие пропусков

In [4]:
num_of_obj = len(features_edit) #количество матчей
filled_rows = features_edit.count() #количество не пустых строк для каждого признака
filled_rows[filled_rows < num_of_obj] #вывод признаков, содержащих пустые строки

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

#### 3. Замена всех пропусков на нули

In [5]:
features_edit = features_edit.fillna(int(0))
X = np.array(features_edit) #создание матрицы объекты-признаки 

#### 4. Целевая переменная

Целевой переменной соответствует столбец radiant_win. Если radiant_win принимает значение 1, значит команда Radiant одержала победу, если 0, то команда Radiant проиграла, а победила команда Dire.

In [6]:
y = np.array(features.radiant_win) #создаем целевой вектор

#### 5. Обучение модели и оценка качества градиентного бустинга с помощью кросс-валидации

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

grid = {'n_estimators' : [10,20,30]}

kf = KFold(n_splits = 5, shuffle = True, random_state = 1)
gbc = GradientBoostingClassifier(random_state = 42,
                                     learning_rate = 0.5)
gs = GridSearchCV(gbc, grid, scoring = 'roc_auc', cv = kf)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.5,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                       

In [9]:
print('Best C =', gs.best_params_['n_estimators'], 'Score:', gs.best_score_)

Best C = 30 Score: 0.702798540711144


In [10]:
#другой способ

from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

#Зададим генератор разбиений выборки по 5 блокам для кросс-валидации
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

quality = {}

for n in range(10, 40, 10):
    gbc = GradientBoostingClassifier(random_state = 42,
                                     learning_rate = 0.5,
                                     n_estimators = n)
    quality[n] = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        gbc.fit(X_train, y_train)
        y_pred = gbc.predict_proba(X_test)
        q = roc_auc_score(y_test, y_pred[:, 1])
        quality[n].append(q)

In [11]:
for key, value in quality.items():
    print('Number of trees:', key, ';', 'Quality:', sum(value)/5)

Number of trees: 10 ; Quality: 0.6852895012128354
Number of trees: 20 ; Quality: 0.6977543950750376
Number of trees: 30 ; Quality: 0.702798540711144


#### Оценим длительность проведения кросс-валидации для градиентного бустинга с 30 деревьями.

In [15]:
gbc = GradientBoostingClassifier(n_estimators = 30,
                                 random_state = 42,
                                 learning_rate = 0.5)
quality = []

In [16]:
import time
import datetime

start_time = datetime.datetime.now()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict_proba(X_test)
    q = roc_auc_score(y_test, y_pred[:, 1])
    quality.append(q)
    
print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:02:16.317267


In [17]:
print('Качество модели:', sum(quality)/5)

Качество модели: 0.702798540711144


Попробуем увеличить число деревьев.

In [90]:
gbc = GradientBoostingClassifier(n_estimators = 40,
                                 random_state = 42,
                                 learning_rate = 0.5)
quality = []

import time
import datetime

start_time = datetime.datetime.now()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict_proba(X_test)
    q = roc_auc_score(y_test, y_pred[:, 1])
    quality.append(q)
    
print('Time elapsed:', datetime.datetime.now() - start_time)
print('Качество модели:', sum(quality)/5)

Time elapsed: 0:03:05.380184
Качество модели: 0.706125886099495


Увеличим число деревьев в композиции до 80 и оценим время работы, а также качество модели, при неизменных значениях остальных параметров.

In [84]:
gbc = GradientBoostingClassifier(n_estimators = 80,
                                 random_state = 42,
                                 learning_rate = 0.5)
quality = []

import time
import datetime

start_time = datetime.datetime.now()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict_proba(X_test)
    q = roc_auc_score(y_test, y_pred[:, 1])
    quality.append(q)
    
print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:06:17.819708


In [86]:
print('Качество модели:', sum(quality)/5)

Качество модели: 0.7115692841915613


Попробуем уменьшить максимальную глубину дерева до 2 (по умолчанию она равна 3).

In [87]:
gbc = GradientBoostingClassifier(n_estimators = 80,
                                 random_state = 42,
                                 learning_rate = 0.5,
                                 max_depth = 2)
quality = []

import time
import datetime

start_time = datetime.datetime.now()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict_proba(X_test)
    q = roc_auc_score(y_test, y_pred[:, 1])
    quality.append(q)
    
print('Time elapsed:', datetime.datetime.now() - start_time)

Time elapsed: 0:04:12.663836


In [88]:
print('Качество модели:', sum(quality)/5)

Качество модели: 0.7109379314463291


In [92]:
gbc = GradientBoostingClassifier(n_estimators = 200,
                                 random_state = 42,
                                 learning_rate = 0.5,
                                 max_depth = 2)
quality = []

import time
import datetime

start_time = datetime.datetime.now()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    gbc.fit(X_train, y_train)
    y_pred = gbc.predict_proba(X_test)
    q = roc_auc_score(y_test, y_pred[:, 1])
    quality.append(q)
    
print('Time elapsed:', datetime.datetime.now() - start_time)
print('Качество модели:', sum(quality)/5)

Time elapsed: 0:10:27.042146
Качество модели: 0.7188085551968637


# Подход 2: логистическая регрессия

In [18]:
import pandas as pd
import numpy as np
#1. Чтение таблицы с признаками
features = pd.read_csv('features.csv', index_col = 'match_id')
features.head(5)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [19]:
#Удаление из таблицы признаков, связанных с итогами матча
features_edit = features.drop(columns = ['duration',
                             'radiant_win',
                             'tower_status_radiant',
                             'tower_status_dire',
                             'barracks_status_dire',
                             'barracks_status_radiant'])

features_edit = features_edit.fillna(int(0)) #замена пропусков 
X = np.array(features_edit) #создание матрицы объекты-признаки 

In [20]:
#Стандартизация признаков

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = np.array(features.radiant_win) #создаем целевой вектор

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [22]:
grid = {'C' : np.power(10.0, np.arange(-5,6))}

kf = KFold(n_splits = 5, shuffle = True, random_state = 1)
lr = LogisticRegression(random_state = 0, 
                             penalty = 'l2')
gs = GridSearchCV(lr, grid, scoring = 'roc_auc', cv = kf)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [23]:
print('Best C =', gs.best_params_['C'], 'Score:', gs.best_score_)

Best C = 0.01 Score: 0.7163753636379495


Другой способ: посмотрим значения от 0.001 до 0.02, а не степень 10.

In [25]:
#Зададим генератор разбиений выборки по 5 блокам для кросс-валидации
cv = KFold(n_splits = 5, shuffle = True, random_state = 1)

quality = {}

#будем подбирать параметр регуляризации C
for c in range(1, 21):
    c = c/1000
    #задание параметром модели
    clf = LogisticRegression(random_state = 0, 
                             penalty = 'l2', 
                             C = c)
                             #solver = 'sag')
    quality[c] = []
    #кросс-валидация
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = clf.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        q = roc_auc_score(y_test, y_pred[:, 1]) #оценка качества модели
        quality[c].append(q)
    print('C:', c, ';', 'Quality:', sum(quality[c])/5)

C: 0.001 ; Quality: 0.7162098751438003
C: 0.002 ; Quality: 0.7163561276366357
C: 0.003 ; Quality: 0.7163797492701399
C: 0.004 ; Quality: 0.7163851302481457
C: 0.005 ; Quality: 0.7163862270516812
C: 0.006 ; Quality: 0.7163848281343317
C: 0.007 ; Quality: 0.7163824932083193
C: 0.008 ; Quality: 0.7163792897370229
C: 0.009 ; Quality: 0.7163768987449377
C: 0.01 ; Quality: 0.7163753636379495
C: 0.011 ; Quality: 0.7163749158601901
C: 0.012 ; Quality: 0.7163730289820891
C: 0.013 ; Quality: 0.716370650510284
C: 0.014 ; Quality: 0.71636942413289
C: 0.015 ; Quality: 0.7163678550093431
C: 0.016 ; Quality: 0.7163669508098626
C: 0.017 ; Quality: 0.7163660591453774
C: 0.018 ; Quality: 0.7163653417456836
C: 0.019 ; Quality: 0.7163651362001676
C: 0.02 ; Quality: 0.7163649543200845


Качество при разных C отличается только по пятому знаку после запятой.
Но наилучшим обладает модель с параметром C равным 0.005.

In [26]:
#оценим время выполнения
start_time = datetime.datetime.now()
clf = LogisticRegression(random_state = 0, 
                             penalty = 'l2', 
                             C = 0.005)
                             #solver = 'sag')
q = []
    #кросс-валидация
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = clf.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    q.append(roc_auc_score(y_test, y_pred[:, 1]))
print('C:', c, ';', 'Quality:', sum(q)/5)

print('Time elapsed:', datetime.datetime.now() - start_time)

C: 0.02 ; Quality: 0.7163862270516812
Time elapsed: 0:00:02.529924


2. Уберем категориальные признаки из рассмотрения

In [27]:
features_without_cat = features_edit.drop(columns = ['lobby_type',
                             'r1_hero',
                             'r2_hero',
                             'r3_hero',
                             'r4_hero',
                             'r5_hero',
                             'd1_hero',
                             'd2_hero',
                             'd3_hero',
                             'd4_hero',
                             'd5_hero'])

#стандартизация
X_new = np.array(features_without_cat)
scaler1 = StandardScaler()
X_new = scaler1.fit_transform(X_new)

In [28]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 1)

quality = {}

for c in range(1, 21):
    c = c/1000
    clf = LogisticRegression(random_state = 0, 
                             penalty = 'l2', 
                             C = c)
                             #solver = 'sag')
    quality[c] = []
    for train_index, test_index in kf.split(X_new):
        X_train, X_test = X_new[train_index], X_new[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = clf.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        q = roc_auc_score(y_test, y_pred[:, 1])
        quality[c].append(q)
    print('C:', c, ';', 'Quality:', sum(quality[c])/5)

C: 0.001 ; Quality: 0.7162333928261827
C: 0.002 ; Quality: 0.7163881448811603
C: 0.003 ; Quality: 0.7164129510923758
C: 0.004 ; Quality: 0.7164196490895375
C: 0.005 ; Quality: 0.7164183249149408
C: 0.006 ; Quality: 0.7164164371314748
C: 0.007 ; Quality: 0.716415091587481
C: 0.008 ; Quality: 0.7164125597798722
C: 0.009 ; Quality: 0.7164112285107282
C: 0.01 ; Quality: 0.7164096834747056
C: 0.011 ; Quality: 0.7164069410777487
C: 0.012 ; Quality: 0.7164057342396586
C: 0.013 ; Quality: 0.7164045143767012
C: 0.014 ; Quality: 0.7164036148272029
C: 0.015 ; Quality: 0.7164014687739002
C: 0.016 ; Quality: 0.7164015561486563
C: 0.017 ; Quality: 0.7164003549261285
C: 0.018 ; Quality: 0.7163998464187598
C: 0.019 ; Quality: 0.7163998447693192
C: 0.02 ; Quality: 0.7163982408968448


In [29]:
#получим все различные идентификаторы для каждого из игроков
r1_hero = list(features.r1_hero.unique())
r2_hero = list(features.r2_hero.unique())
r3_hero = list(features.r3_hero.unique())
r4_hero = list(features.r4_hero.unique())
r5_hero = list(features.r5_hero.unique())
d1_hero = list(features.d1_hero.unique())
d2_hero = list(features.d2_hero.unique())
d3_hero = list(features.d3_hero.unique())
d4_hero = list(features.d4_hero.unique())
d5_hero = list(features.d5_hero.unique())

#число уникальных идентификаторов

heroes = set(r1_hero + r2_hero + r3_hero + r4_hero + r5_hero + d1_hero + d2_hero + d3_hero + d4_hero + d5_hero)
print(max(heroes))

112


In [30]:
#из них встречаются
len(heroes)

108

In [31]:
#создание "мешка слов"
N = max(heroes)
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [32]:
X_train = np.hstack((X_new,X_pick))

In [33]:
grid = {'C' : np.power(10.0, np.arange(-3,3))}

kf = KFold(n_splits = 5, shuffle = True, random_state = 1)
lr = LogisticRegression(random_state = 2, 
                             penalty = 'l2',
                             solver = 'newton-cg')
gs = GridSearchCV(lr, grid, scoring = 'roc_auc', cv = kf)
gs.fit(X_train, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=2, solver='newton-cg',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [34]:
print('Best C =', gs.best_params_['C'], 'Score:', gs.best_score_)

Best C = 0.1 Score: 0.7518728282182467


Лучшим качеством обладает модель логистической регрессии с параметром регуляризации C = 0.1 обученная на выборке с "мешком слов". Для дальнейшего предсказания вероятностей победы команды Radiant для тестовой выборки будет использована эта модель. В качестве решателя выбран Newton-CG

In [35]:
#"Натренируем" модель с параметром регуляризации С = 0.01 на признаках с "мешком слов"
best_model = LogisticRegression(random_state = 2, 
                                penalty = 'l2',
                                C = 0.1,
                                solver = 'newton-cg')
best_model.fit(X_train, y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
#предобработка таблицы признаков тестовой выборки
features_test = pd.read_csv('features_test.csv', index_col = 'match_id')
#удаление категориальных признаков 
features_test_edit = features_test.drop(columns = ['lobby_type',
                             'r1_hero',
                             'r2_hero',
                             'r3_hero',
                             'r4_hero',
                             'r5_hero',
                             'd1_hero',
                             'd2_hero',
                             'd3_hero',
                             'd4_hero',
                             'd5_hero'])
#заполнение пропусков 
features_test_edit = features_test_edit.fillna(0)
#создание матрицы объекты-признаки
X_test = np.array(features_test_edit)
#стандартизация числовых признаков тестовой выборки
X_test = scaler1.transform(X_test)
#создание мешка слов
N = max(heroes)
X_pick_test = np.zeros((features_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick_test[i, features_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
#матрица объекты-признаки    
X_test = np.hstack((X_test, X_pick_test))     

In [39]:
#предсказание
y_pred = best_model.predict_proba(X_test)[:, 1]
#оценим вероятности визуально: они различны 
print(y_test)

[0.82390429 0.75299146 0.18805589 ... 0.23450857 0.62508283 0.42605145]


In [40]:
if len(y_pred) == sum(y_pred <= 1) and len(y_pred) == sum(y_test >= 0):
    print('Все вероятности находятся в интервале [0,1]')

Все вероятности находятся в интервале [0,1]


In [42]:
print('Наибольшая вероятность победы Radiant:', round(max(y_pred), 3))
print('Наименьшая вероятность победы Radiant:', round(min(y_pred), 3))

Наибольшая вероятность победы Radiant: 0.996
Наименьшая вероятность победы Radiant: 0.008
